In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import CMR_IA as cmr
import sys

sys.path.append("../../Modeling/fitting")
from optimization_utils import make_boundary
from object_funcs import anal_perform_6b as anal_perform

## Load Stimuli and Semantic Matrix

In [2]:
with open("simu6b_data/simu6b_design.pkl", "rb") as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
# df_study = df_study.loc[df_study.session < 100]  # for testing
# df_test = df_test.loc[df_test.session < 100]

In [3]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,list,session
0,120,459,BELT,DIAPER,0,0
1,1315,1260,SKETCH,SEAFOOD,0,0
2,357,723,COMPANION,HOSTESS,0,0
3,1551,1202,VEAL,ROCKET,0,0
4,1081,1066,PLANE,PIGMENT,0,0
...,...,...,...,...,...,...
71995,306,793,CHOCOLATE,LADDER,5,999
71996,846,1117,LOVER,POWDER,5,999
71997,1102,1048,PONY,PERCH,5,999
71998,658,903,GRAVE,MOAT,5,999


In [4]:
df_test

,test_itemno1,test_itemno2,test_item,correct_ans,order,rep,test,list,session,test_pos,pair_idx
0,51,-1,ASTRONAUT,1083,2,1,1,0,0,1,8
1,1202,-1,ROCKET,1551,2,1,1,0,0,2,4
2,979,-1,OPERATOR,142,1,1,1,0,0,3,11
3,134,-1,BLADE,105,1,1,1,0,0,4,9
4,1081,-1,PLANE,1066,1,1,1,0,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...
143995,326,-1,CLOSET,90,2,1,2,5,999,20,71995
143996,658,-1,GRAVE,903,1,1,2,5,999,21,71999
143997,162,-1,BOYFRIEND,769,1,1,2,5,999,22,72000
143998,1172,-1,REBEL,1139,2,1,2,5,999,23,71992


In [5]:
s_mat = np.load("../wordpools/ltp_FR_similarity_matrix.npy")

## Run CMR-IA

In [6]:
# define parameters
params = cmr.make_default_params()
params.update(learn_while_retrieving=True, nitems_in_accumulator=96, use_new_context=True)

In [7]:
# load pso results
_, _, what_to_fit = make_boundary(sim_name="6b")
optim_params = np.loadtxt("simu6b_data/6b_240529_100-100.txt")
for pname, pvalue in zip(what_to_fit, optim_params):
    params[pname] = pvalue
params

{'beta_enc': 0.953602,
 'beta_rec': 0.519641,
 'beta_cue': 0.854426,
 'beta_rec_post': 0.372024,
 'beta_distract': 0.830854,
 'phi_s': 2.17525,
 'phi_d': 0.731406,
 's_cf': 0.051149,
 's_fc': 0.148548,
 'kappa': 0.035835,
 'eta': 0.014504,
 'omega': 6.157514,
 'alpha': 0.957752,
 'lamb': 0.030318,
 'c_thresh': 0.301722,
 'c_thresh_itm': 0.5,
 'c_thresh_assoc': 0.5,
 'd_assoc': 1,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 96,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'use_new_context': True,
 'psi_s': 0,
 'psi_c': 1,
 'c_s': 0,
 'thresh_sigma': 0,
 'ban_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'a': 2800,
 'b': 20,
 'gamma_fc': 0.401852,
 'gamma_cf': 0.541581}

In [8]:
# run the model
df_simu, f_in, f_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat, mode="CR-CR")
df_simu["test_pos"] = np.tile(np.arange(1, 25), 6000)

# merge to get more info
df_simu = df_simu.merge(df_test, on=["session", "list", "test_itemno1", "test_itemno2", "test_pos"])
df_simu["correct"] = df_simu.s_resp == df_simu.correct_ans
df_simu

CMR Time: 108.78228282928467


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct
0,0,0,51,-1,1083.0,2020.0,0.534844,1,ASTRONAUT,1083,2,1,1,8,True
1,0,0,1202,-1,1551.0,2380.0,0.557797,2,ROCKET,1551,2,1,1,4,True
2,0,0,979,-1,142.0,2640.0,0.539726,3,OPERATOR,142,1,1,1,11,True
3,0,0,134,-1,105.0,2640.0,0.588176,4,BLADE,105,1,1,1,9,True
4,0,0,1081,-1,1066.0,1970.0,0.577245,5,PLANE,1066,1,1,1,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143995,999,5,326,-1,909.0,1040.0,0.338011,20,CLOSET,90,2,1,2,71995,False
143996,999,5,658,-1,1544.0,1360.0,0.377957,21,GRAVE,903,1,1,2,71999,False
143997,999,5,162,-1,1063.0,1320.0,0.310407,22,BOYFRIEND,769,1,1,2,72000,False
143998,999,5,1172,-1,688.0,1560.0,0.372899,23,REBEL,1139,2,1,2,71992,False


In [9]:
# get condition and congruence
df_cond = df_simu.groupby(["pair_idx", "test"])["order"].mean().to_frame(name="corr_rate").reset_index()
df_cond = df_cond.pivot_table(index="pair_idx", columns="test", values="corr_rate").reset_index()
df_cond.columns = ["pair_idx", "test1", "test2"]


def cond(x):
    test1 = x["test1"]
    test2 = x["test2"]
    if test1 == 1 and test2 == 1:
        return "F-F"
    elif test1 == 1 and test2 == 2:
        return "F-B"
    elif test1 == 2 and test2 == 1:
        return "B-F"
    elif test1 == 2 and test2 == 2:
        return "B-B"


df_cond["cond"] = df_cond.apply(lambda x: cond(x), axis=1)
df_cond["cong"] = df_cond.apply(lambda x: "Identical" if x["cond"] == "F-F" or x["cond"] == "B-B" else "Reversed", axis=1)
df_cond

,pair_idx,test1,test2,cond,cong
0,1,1.0,1.0,F-F,Identical
1,2,2.0,1.0,B-F,Reversed
2,3,1.0,1.0,F-F,Identical
3,4,2.0,1.0,B-F,Reversed
4,5,1.0,2.0,F-B,Reversed
...,...,...,...,...,...
71995,71996,2.0,1.0,B-F,Reversed
71996,71997,1.0,1.0,F-F,Identical
71997,71998,2.0,2.0,B-B,Identical
71998,71999,2.0,1.0,B-F,Reversed


In [10]:
# merge condition and congruence
pairidx2cond = df_cond.loc[:, ["pair_idx", "cond"]].set_index("pair_idx").to_dict()["cond"]
pairidx2cong = df_cond.loc[:, ["pair_idx", "cong"]].set_index("pair_idx").to_dict()["cong"]
df_simu["cond"] = df_simu.apply(lambda x: pairidx2cond[x["pair_idx"]], axis=1)
df_simu["cong"] = df_simu.apply(lambda x: pairidx2cong[x["pair_idx"]], axis=1)
df_simu.head(24)

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong
0,0,0,51,-1,1083.0,2020.0,0.534844,1,ASTRONAUT,1083,2,1,1,8,True,B-B,Identical
1,0,0,1202,-1,1551.0,2380.0,0.557797,2,ROCKET,1551,2,1,1,4,True,B-F,Reversed
2,0,0,979,-1,142.0,2640.0,0.539726,3,OPERATOR,142,1,1,1,11,True,F-F,Identical
3,0,0,134,-1,105.0,2640.0,0.588176,4,BLADE,105,1,1,1,9,True,F-B,Reversed
4,0,0,1081,-1,1066.0,1970.0,0.577245,5,PLANE,1066,1,1,1,5,True,F-B,Reversed
5,0,0,357,-1,723.0,1340.0,0.585341,6,COMPANION,723,1,1,1,3,True,F-F,Identical
6,0,0,831,-1,1435.0,3150.0,0.543778,7,LITERATURE,1435,2,1,1,7,True,B-B,Identical
7,0,0,1188,-1,641.0,3520.0,0.531187,8,RIB,641,2,1,1,12,True,B-F,Reversed
8,0,0,1260,-1,1315.0,1110.0,0.565651,9,SEAFOOD,1315,2,1,1,2,True,B-F,Reversed
9,0,0,12,-1,226.0,2690.0,0.581178,10,AIRPLANE,226,1,1,1,10,True,F-B,Reversed


In [11]:
# get f_in
sessions = np.unique(df_simu.session.to_numpy())
tmp_corr_fin = []
tmp_omean_fin = []
tmp_omax_fin = []

for sess in sessions:
    df_tmp = df_study.loc[df_study.session == sess]
    tmp1 = df_tmp.study_itemno1.to_numpy()
    tmp2 = df_tmp.study_itemno2.to_numpy()
    tmp = np.concatenate((tmp1, tmp2))
    tmp = np.sort(tmp)

    df_tmp1 = df_simu.loc[df_simu.session == sess]
    tmp_test = df_tmp1.test_itemno1
    tmp_corr = df_tmp1.correct_ans
    # tmp_test = df_simu.loc[df_simu.session == sess, 'test_itemno']
    # tmp_corr = df_simu.loc[df_simu.session == sess, 'correct_ans']
    testid = np.searchsorted(tmp, tmp_test)
    corrid = np.searchsorted(tmp, tmp_corr)

    fin_sess = f_dif[sess]
    # tmp_corr_fin = []
    # tmp_omean_fin = []
    # tmp_omax_fin = []
    for i in range(len(testid)):
        tmp_corr_fin.append(fin_sess[i][corrid[i]])
        tmp_omean_fin.append(np.mean(np.delete(fin_sess[i], corrid[i])))
        tmp_omax_fin.append(np.delete(fin_sess[i], corrid[i]).max())
    # df_simu.loc[df_simu.session == sess, 'corr_fin']= tmp_corr_fin
    # df_simu.loc[df_simu.session == sess, 'omean_fin'] = tmp_omean_fin
    # df_simu.loc[df_simu.session == sess, 'omax_fin'] = tmp_omax_fin

df_simu["corr_fin"] = tmp_corr_fin
df_simu["omean_fin"] = tmp_omean_fin
df_simu["omax_fin"] = tmp_omax_fin
df_simu

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong,corr_fin,omean_fin,omax_fin
0,0,0,51,-1,1083.0,2020.0,0.534844,1,ASTRONAUT,1083,2,1,1,8,True,B-B,Identical,-0.442592,-0.993926,-0.709784
1,0,0,1202,-1,1551.0,2380.0,0.557797,2,ROCKET,1551,2,1,1,4,True,B-F,Reversed,-0.318100,-1.065843,-0.739677
2,0,0,979,-1,142.0,2640.0,0.539726,3,OPERATOR,142,1,1,1,11,True,F-F,Identical,-0.435481,-1.149513,-0.768745
3,0,0,134,-1,105.0,2640.0,0.588176,4,BLADE,105,1,1,1,9,True,F-B,Reversed,-0.407076,-1.212024,-0.663599
4,0,0,1081,-1,1066.0,1970.0,0.577245,5,PLANE,1066,1,1,1,5,True,F-B,Reversed,-0.357837,-1.282700,-0.702991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143995,999,5,326,-1,909.0,1040.0,0.338011,20,CLOSET,90,2,1,2,71995,False,F-B,Reversed,-0.074457,-1.079968,0.265874
143996,999,5,658,-1,1544.0,1360.0,0.377957,21,GRAVE,903,1,1,2,71999,False,B-F,Reversed,-0.053178,-1.157051,0.413414
143997,999,5,162,-1,1063.0,1320.0,0.310407,22,BOYFRIEND,769,1,1,2,72000,False,B-F,Reversed,-0.056332,-1.201241,0.302097
143998,999,5,1172,-1,688.0,1560.0,0.372899,23,REBEL,1139,2,1,2,71992,False,F-B,Reversed,0.016244,-1.276167,0.054145


In [12]:
df_fin = df_simu.groupby(["cong", "test"])[["corr_fin", "omean_fin", "omax_fin"]].mean().reset_index()
df_fin["dif"] = df_fin["corr_fin"] - df_fin["omax_fin"]
df_fin

,cong,test,corr_fin,omean_fin,omax_fin,dif
0,Identical,1,-0.245470,-1.143712,0.149977,-0.395447
1,Identical,2,-0.065013,-1.093350,0.691608,-0.756620
2,Reversed,1,-0.248333,-1.142588,0.153390,-0.401723
3,Reversed,2,0.099221,-1.121895,0.388524,-0.289303


In [13]:
df_simu.to_pickle("simu6b_data/simu6b_result.pkl")

## Analysis

In [14]:
with open("simu6b_data/simu6b_result.pkl", "rb") as inp:
    df_simu = pickle.load(inp)

In [15]:
# anal_perform used by pso
subjects = np.unique(df_simu.session)
inde_stats = []
reve_stats = []
for subj in subjects:
    df_subj_inde = df_simu.query(f"session == {subj} and cong == 'Identical'").copy()
    inde_stats.append(list(anal_perform(df_subj_inde)))

    df_subj_reve = df_simu.query(f"session == {subj} and cong == 'Reversed'").copy()
    reve_stats.append(list(anal_perform(df_subj_reve)))

In [16]:
np.array(inde_stats)

array([[0.25      , 0.05555556, 0.02777778, 0.66666667, 0.96828753],
       [0.36111111, 0.02777778, 0.02777778, 0.58333333, 0.98461538],
       [0.25      , 0.08333333, 0.08333333, 0.58333333, 0.88683603],
       ...,
       [0.22222222, 0.02777778, 0.        , 0.75      , 0.99360341],
       [0.25      , 0.        , 0.02777778, 0.72222222, 0.99405941],
       [0.25      , 0.        , 0.08333333, 0.66666667, 0.98507463]])

In [17]:
np.array(reve_stats)

array([[0.27777778, 0.02777778, 0.16666667, 0.52777778, 0.90909091],
       [0.36111111, 0.02777778, 0.11111111, 0.5       , 0.94736842],
       [0.25      , 0.02777778, 0.13888889, 0.58333333, 0.92235294],
       ...,
       [0.16666667, 0.        , 0.11111111, 0.72222222, 0.97421203],
       [0.25      , 0.05555556, 0.11111111, 0.58333333, 0.89559165],
       [0.25      , 0.        , 0.11111111, 0.63888889, 0.98004435]])

In [18]:
np.mean(inde_stats, axis=0)

array([0.30758333, 0.02522222, 0.02011111, 0.64708333, 0.98648674])

In [19]:
np.mean(reve_stats, axis=0)

array([0.30841667, 0.02611111, 0.11322222, 0.55225   , 0.94018854])

In [20]:
inde_stats_mean = np.mean(inde_stats, axis=0)
reve_stats_mean = np.mean(reve_stats, axis=0)
inde_ground_truth = np.array([0.319, 0.006, 0.012, 0.663, 0.94])
reve_ground_truth = np.array([0.293, 0.049, 0.122, 0.537, 0.96])
err = (np.mean(np.power(inde_stats_mean - inde_ground_truth, 2)) + np.mean(np.power(reve_stats_mean - reve_ground_truth, 2))) / 2
err

0.0004443662396477719